In [1]:
import arcpy
import csv
import os

# === Paths ===
gdb_path = r"X:\Projects\BetterRoads\Sidewalks\Schema_Update_ProProject\Schema_Update_ProProject.gdb"
output_dir = os.path.dirname(gdb_path)

# === Output CSV paths ===
domains_csv = os.path.join(output_dir, "domains.csv")
fields_csv = os.path.join(output_dir, "fields.csv")
relationships_csv = os.path.join(output_dir, "relationships.csv")

# === Domains ===
domains = arcpy.da.ListDomains(gdb_path)
with open(domains_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Domain Name", "Domain Type", "Field Type", "Code or Range", "Description"])
    for domain in domains:
        field_type = getattr(domain, "fieldType", "Unknown")
        if domain.domainType == "CodedValue":
            for code, desc in domain.codedValues.items():
                writer.writerow([domain.name, domain.domainType, field_type, code, desc])
        elif domain.domainType == "Range":
            writer.writerow([domain.name, domain.domainType, field_type,
                             f"{domain.range[0]} - {domain.range[1]}", ""])

# === Feature Classes and Tables (Fields) ===
arcpy.env.workspace = gdb_path
datasets = arcpy.ListFeatureClasses() + arcpy.ListTables()
with open(fields_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Dataset", "Field Name", "Alias", "Type", "Length", "Nullable", "Domain", "Default Value"])
    for dataset in datasets:
        full_path = os.path.join(gdb_path, dataset)
        fields = arcpy.ListFields(full_path)
        for field in fields:
            writer.writerow([
                dataset,
                field.name,
                field.aliasName,
                field.type,
                field.length,
                field.isNullable,
                field.domain,
                field.defaultValue if hasattr(field, "defaultValue") else ""
            ])

# === Relationships ===
desc = arcpy.Describe(gdb_path)
if hasattr(desc, "relationshipClasses"):
    relationships = desc.relationshipClasses
else:
    relationships = []

with open(relationships_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Origin Table", "Destination Table", "Cardinality", "Forward Label", "Backward Label", "Is Attributed"])
    for rel in relationships:
        writer.writerow([
            rel.name,
            rel.originClassNames[0] if rel.originClassNames else "",
            rel.destinationClassNames[0] if rel.destinationClassNames else "",
            rel.cardinality,
            rel.forwardPathLabel,
            rel.backwardPathLabel,
            rel.isAttributed
        ])
